In [1]:
import numpy as np

In [2]:
def min_dist(dist_matrix):
    n, m = dist_matrix.shape
    min_dist = None
    min_i = 0
    min_j = 0
    for i in range(n):
        for j in range(i + 1, m):
            dist = dist_matrix[i, j]
            if dist != 0 and (min_dist is None or dist <= min_dist):
                min_dist = dist
                min_i = i
                min_j = j
    return min_i, min_j

In [3]:
def tree_dist(dist_matrix, i, j, n):
    di = dist_matrix[i, j] / 2 + (dist_matrix[i, :].sum() - dist_matrix[j, :].sum()) / (2 * (n - 2))
    dj = dist_matrix[i, j] - di
    return di, dj

In [4]:
def caluclate_q_matrix(dist_matrix, n):
    mask = (dist_matrix != 0)
    q_matrix = n * dist_matrix - mask * (dist_matrix.sum(axis=0).reshape(1, -1) + dist_matrix.sum(axis=0).reshape(-1, 1))
    return q_matrix

In [5]:
def merge_subtrees(trees, i, j, dl, dr):
    trees = trees.copy()
    childl = trees[i]
    childr = trees[j]
    trees[i] = ((childl, dl), (childr, dr))
    trees[j] = 0
    return trees

In [6]:
def recalculate_distances(dist_matrix, i, j):
    dist_matrix = dist_matrix.copy()
    mask = dist_matrix != 0
    distances_row = (dist_matrix[i, :] + dist_matrix[j, :] - dist_matrix[i, j]) / 2
    distances_col = (dist_matrix[:, i] + dist_matrix[:, j] - dist_matrix[j, i]) / 2
    dist_matrix[i, :] = distances_row
    dist_matrix[:, i] = distances_col
    dist_matrix[j, :] *= 0 
    dist_matrix[:, j] *= 0
    dist_matrix = mask * dist_matrix
    
    return dist_matrix

In [7]:
def to_newick(tree):
    def recurse(tree):
        if isinstance(tree, tuple):
            (childl, distl), (childr, distr) = tree
            childl = recurse(childl)
            childr = recurse(childr)
            return f"({childl}:{distl :.2f},{childr}:{distr :.2f})"
        else:
            node = tree
            return f"{node}"
        
    (childl, distl), (childr, distr) = tree
    childl = recurse(childl)
    childr = recurse(childr)
    return f"({childl}:{distl :.2f},{childr}:{distr :.2f});"

В качестве корня выбирается виртуальная вершина, разделяющая ребро на две части.

In [8]:
def nj(trees, dist_matrix):
    steps = len(trees)
    while steps >= 3:
        i, j = min_dist(caluclate_q_matrix(dist_matrix, steps))
        di, dj = tree_dist(dist_matrix, i, j, steps)
        dist_matrix = recalculate_distances(dist_matrix, i, j)
        trees = merge_subtrees(trees, i, j, di, dj)
        steps -= 1
    i, j = min_dist(dist_matrix)
    trees = merge_subtrees(trees, i, j, dist_matrix[i, j] / 2, dist_matrix[i, j] / 2)
    tree = trees[i]
    return tree

# Тестовые данные

In [9]:
test_1_trees = ['A', 'B', 'C', 'D']
test_1_dist = np.array([[0, 16, 16, 10],
                        [16, 0,  8,  8],
                        [16, 8, 0,   4],
                        [10, 8, 4,   0]
                    ], dtype=np.float64)

In [10]:
test_2_trees = ['A', 'B', 'C', 'D', 'E', 'F']
test_2_dist = np.array([[0, 5,  4, 7,  6,  8],
                        [5, 0,  7, 10, 9, 11],
                        [4, 7,  0, 7,  6,  8],
                        [7, 10, 7, 0,  5,  9],
                        [6, 9,  6, 5,  0,  8],
                        [8, 11, 8, 9,  8,  0]
                       ], dtype=np.float64)

# Проверка NJ

## Тест 1

In [11]:
to_newick(nj(test_1_trees, test_1_dist))

'(A:5.25,(B:5.50,(C:3.50,D:0.50):0.50):5.25);'

In [12]:
to_newick(nj(test_1_trees, test_1_dist))

'(A:5.25,(B:5.50,(C:3.50,D:0.50):0.50):5.25);'

## Teст 2

In [13]:
to_newick(nj(test_2_trees, test_2_dist))

'((((A:1.00,B:4.00):1.00,C:2.00):1.00,(D:3.00,E:2.00):1.00):2.50,F:2.50);'

### Пример из википедии

In [14]:
test_trees = ['A', 'B', 'C', 'D', 'E']
test_dist = np.array([[0, 5,  9,  9,  8],
                        [5, 0,  10, 10, 9],
                        [9, 10, 0,  8,  7],
                        [9, 10, 8,  0,  3],
                        [8, 9,  7,  3,  0],
                       ], dtype=np.float64)

In [15]:
to_newick(nj(test_trees, test_dist))

'(((A:2.00,B:3.00):3.00,(D:2.00,E:1.00):2.00):2.00,C:2.00);'

### Пример из статьи

Статья: [The Neighbor-joining Method: A New Method for Reconstructing Phylogenetic Trees](https://pdfs.semanticscholar.org/16ec/b957cd06767c666c9f46c41f506a0bf6a59b.pdf?_ga=2.79234768.536811194.1606479032-1249458473.1606479032)

In [16]:
test_trees = ['"1"', '"2"', '"3"', '"4"', '"5"', '"6"', '"7"', '"8"']
test_dist = np.array([[0,  7,  8,  11,  13,  16, 13, 17],
                      [7,  0,  5,  8,   10,  13, 10, 14],
                      [8,  5,  0,  5,   7,   10, 7,  11],
                      [11, 8,  5,  0,   8,   11, 8,  12],
                      [13, 10, 7,  8,   0,   5,  6,  10],
                      [16, 13, 10, 11,  5,   0,  9,  13],
                      [13, 10, 7,  8,   6,   9,  0,  8],
                      [17, 14, 11, 12,  10,  13, 8,  0]
                    ], dtype=np.float64)

In [17]:
to_newick(nj(test_trees, test_dist))

'(((((("1":5.00,"2":2.00):2.00,"3":1.00):1.00,"4":3.00):2.00,("5":1.00,"6":4.00):2.00):1.00,"7":2.00):3.00,"8":3.00);'